In [ ]:
# 새 셀에서 실행
print("📋 TOP 30 데이터 컬럼 확인:")
print(f"TOP 30 컬럼: {list(top30_df.columns)}")
print(f"\ncoords_df 컬럼: {list(coords_df.columns)}")

# 상권 관련 컬럼 찾기
top30_district_cols = [col for col in top30_df.columns if '상권' in col]
coords_district_cols = [col for col in coords_df.columns if '상권' in col]

print(f"\nTOP 30 상권 관련 컬럼: {top30_district_cols}")
print(f"coords_df 상권 관련 컬럼: {coords_district_cols}")

In [ ]:
# 🔗 TOP 30과 좌표 데이터 조인 (수정)
print("🔗 TOP 30과 좌표 데이터 조인 (수정)")

# 조인 방법 결정
if '상권_코드' in top30_df.columns and '상권_코드' in coords_df.columns:
    # 상권_코드로 조인
    top30_with_coords = top30_df.merge(
        coords_df[['상권_코드', 'latitude', 'longitude']], 
        on='상권_코드', how='left'
    )
    print("✅ 상권_코드로 조인 완료")
elif '상권_코드_명' in top30_df.columns and '상권_코드_명' in coords_df.columns:
    # 상권_코드_명으로 조인 ← 이 경로로 실행될 예정!
    top30_with_coords = top30_df.merge(
        coords_df[['상권_코드_명', 'latitude', 'longitude']], 
        on='상권_코드_명', how='left'
    )
    print("✅ 상권_코드_명으로 조인 완료")
else:
    # 수동 매핑 사용
    print("🔄 수동 좌표 매핑 사용")
    # ... (수동 매핑 코드)

# 매칭 현황 확인
coord_matched = top30_with_coords['latitude'].notna().sum()
print(f"📊 좌표 매칭 현황: {coord_matched}/30 ({coord_matched/30:.1%})")

In [ ]:
# 🗺️ 곱창집 입지 분석 지도 시각화 (수정된 버전)
# 04_map_visualization_fixed.ipynb
#
# 목적: TOP 30 곱창집 최적 입지를 인터랙티브 지도에서 시각화
# 수정: 조인 오류 해결, 수동 좌표 매핑 우선 적용
# ================================================================================

# 📚 라이브러리 임포트
# ================================================================
import pandas as pd
import numpy as np
from pathlib import Path
import folium
from folium import plugins
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🗺️ 곱창집 입지 분석 지도 시각화 (수정된 버전) 시작!")
print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)

# 📂 경로 설정 및 데이터 로드
# ================================================================
OUTPUT_PATH = Path("../docs")

# TOP 30 데이터 로드
top30_file = OUTPUT_PATH / "gopchang_top30_locations_v2.csv"
if top30_file.exists():
    top30_df = pd.read_csv(top30_file, encoding='utf-8-sig')
    print(f"✅ TOP 30 데이터 로드: {top30_df.shape}")
    print(f"📋 컬럼: {list(top30_df.columns)}")
else:
    print("❌ TOP 30 파일을 찾을 수 없습니다.")
    print("💡 먼저 03-1_kpi_scoring_fixed.ipynb를 실행해주세요!")

print("=" * 70)

# 🗺️ 주요 상권 좌표 수동 매핑 (가장 확실한 방법)
# ================================================================
print("\n🗺️ 주요 상권 좌표 수동 매핑")
print("-" * 20)

# TOP 30에 포함된 모든 상권의 좌표
manual_coords = {
    '강남역': (37.4979, 127.0276),
    '종로?청계 관광특구': (37.5703, 126.9910),
    '명동 남대문 북창동 다동 무교동 관광특구': (37.5636, 126.9831),
    '신촌역(신촌역, 신촌로터리)': (37.5547, 126.9366),
    '종로3가역': (37.5703, 126.9910),
    '홍대입구역(홍대)': (37.5568, 126.9236),
    '노량진역(노량진)': (37.5139, 126.9425),
    '노원역': (37.6542, 127.0621),
    '잠실 관광특구': (37.5133, 127.1028),
    '국회의사당역(국회의사당)': (37.5290, 126.9173),
    '연남동(홍대)': (37.5602, 126.9264),
    '까치산역 3번': (37.5317, 126.8456),
    '남영동 먹자골목': (37.5409, 126.9710),
    '가산디지털단지': (37.4816, 126.8827),
    '수유역': (37.6378, 127.0257),
    '신림역(신림)': (37.4844, 126.9298),
    '영등포역(영등포)': (37.5158, 126.9076),
    '망리단길': (37.5547, 126.9062),
    '대학로(혜화역)': (37.5820, 126.9996),
    '불광역 7번': (37.6108, 126.9329),
    '선릉역': (37.5044, 127.0490),
    '북창동(시청역_6번)': (37.5640, 126.9769),
    '총신대입구역(이수, 총신대)': (37.4865, 126.9817),
    '신논현역': (37.5044, 127.0251),
    '길동역': (37.5370, 127.1441),
    '역삼역': (37.5000, 127.0364),
    '여의도역(여의도)': (37.5213, 126.9245),
    '서울대입구역': (37.4813, 126.9527),
    '교대역(법원.검찰청)': (37.4942, 127.0134),
    '건대입구역(건대)': (37.5405, 127.0705)
}

print(f"📍 수동 매핑 좌표: {len(manual_coords)}개 상권")

# TOP 30에 좌표 직접 추가
map_data = top30_df.copy()
map_data['latitude'] = map_data['상권_코드_명'].map(
    lambda x: manual_coords.get(x, (None, None))[0]
)
map_data['longitude'] = map_data['상권_코드_명'].map(
    lambda x: manual_coords.get(x, (None, None))[1]
)

# 좌표 매칭 현황
coord_matched = map_data['latitude'].notna().sum()
print(f"📊 좌표 매칭 현황: {coord_matched}/30 ({coord_matched/30:.1%})")

# 좌표 없는 상권 확인
missing_coords = map_data[map_data['latitude'].isna()]
if len(missing_coords) > 0:
    print(f"\n❌ 좌표 없는 상권들:")
    for idx, row in missing_coords.iterrows():
        print(f"   • {row['상권_코드_명']}")
else:
    print("✅ 모든 상권 좌표 매핑 완료!")

# 좌표 있는 상권만 필터링 (지도 표시용)
map_data = map_data[map_data['latitude'].notna()].reset_index(drop=True)
print(f"🗺️ 지도 표시 대상: {len(map_data)}개 상권")

print("=" * 70)

# 🎨 지도 스타일링 설정
# ================================================================
print("\n🎨 지도 스타일링 설정")
print("-" * 15)

def get_color_by_score(score):
    """적합도 점수에 따른 색상 반환"""
    if score >= 40:
        return 'red'        # 최우수 (40점 이상)
    elif score >= 35:
        return 'orange'     # 우수 (35-40점)
    elif score >= 30:
        return 'yellow'     # 양호 (30-35점)
    elif score >= 25:
        return 'green'      # 보통 (25-30점)
    else:
        return 'blue'       # 기타 (25점 미만)

def get_marker_size(score, base_size=8):
    """적합도 점수에 따른 마커 크기 반환"""
    return base_size + (score - 20) * 0.4

# 색상 및 크기 컬럼 추가
map_data['color'] = map_data['곱창집_적합도_v2'].apply(get_color_by_score)
map_data['marker_size'] = map_data['곱창집_적합도_v2'].apply(get_marker_size)

print(f"📊 색상별 분포:")
color_counts = map_data['color'].value_counts()
color_info = {
    'red': '최우수 (40점 이상)',
    'orange': '우수 (35-40점)',
    'yellow': '양호 (30-35점)',
    'green': '보통 (25-30점)',
    'blue': '기타 (25점 미만)'
}

for color, count in color_counts.items():
    print(f"   {color} ({color_info[color]}): {count}개")

print("=" * 70)

# 🗺️ 인터랙티브 지도 생성
# ================================================================
print("\n🗺️ 인터랙티브 지도 생성")
print("-" * 15)

# 서울 중심 좌표
seoul_center = [37.5665, 126.9780]

# 지도 객체 생성
m = folium.Map(
    location=seoul_center,
    zoom_start=11,
    tiles='OpenStreetMap',
    width='100%',
    height='600px'
)

print("🎯 TOP 30 마커 추가 중...")

# TOP 30 마커 추가
for idx, row in map_data.iterrows():
    # 순위 계산 (적합도 점수 기준)
    rank = idx + 1
    
    # 팝업 내용 생성
    popup_html = f"""
    <div style="width: 320px; font-family: Arial, sans-serif;">
        <h3 style="color: #d32f2f; margin: 5px 0; text-align: center;">
            🍖 TOP {rank}: {row['상권_코드_명']}
        </h3>
        <hr style="margin: 8px 0;">
        
        <div style="background: linear-gradient(135deg, #ff6b6b, #ee5a52); 
                    color: white; padding: 12px; border-radius: 8px; text-align: center; margin: 8px 0;">
            <h4 style="margin: 0 0 5px 0; font-size: 14px;">📊 곱창집 적합도 점수</h4>
            <div style="font-size: 24px; font-weight: bold;">
                {row['곱창집_적합도_v2']:.1f}점
            </div>
        </div>
        
        <div style="background: #f8f9fa; padding: 10px; border-radius: 6px; margin: 8px 0;">
            <h4 style="color: #1976d2; margin: 0 0 8px 0; font-size: 13px;">🔍 핵심 지표</h4>
            <div style="font-size: 12px; line-height: 1.6;">
                🌙 <b>야간유동인구:</b> {row['T1_야간유동인구']:,}명<br>
                💰 <b>한식매출:</b> {row['한식_월매출']/100000000:.1f}억원<br>
                🏪 <b>한식점포:</b> {row['한식_점포수']:.0f}개<br>
                📊 <b>상권활성도:</b> {row['C2_상권활성도']/100000000:.0f}억원
            </div>
        </div>
        
        <div style="background: #e3f2fd; padding: 8px; border-radius: 6px; margin: 8px 0;">
            <h4 style="color: #1976d2; margin: 0 0 6px 0; font-size: 12px;">⭐ 세부 점수 (0-100점)</h4>
            <div style="font-size: 11px; line-height: 1.4;">
                <div style="display: flex; justify-content: space-between;">
                    <span>T1(야간유동):</span> <b>{row['T1_점수_v2']:.0f}점</b>
                </div>
                <div style="display: flex; justify-content: space-between;">
                    <span>T2(매출밀도):</span> <b>{row['T2_점수_v2']:.0f}점</b>
                </div>
                <div style="display: flex; justify-content: space-between;">
                    <span>C1(경쟁우위):</span> <b>{row['C1_점수_v2']:.0f}점</b>
                </div>
                <div style="display: flex; justify-content: space-between;">
                    <span>C2(상권활성):</span> <b>{row['C2_점수_v2']:.0f}점</b>
                </div>
                <div style="display: flex; justify-content: space-between;">
                    <span>E1(주류친화):</span> <b>{row['E1_점수_v2']:.0f}점</b>
                </div>
            </div>
        </div>
        
        <div style="text-align: center; font-size: 10px; color: #666; margin-top: 8px;">
            💡 클릭하여 상세 정보 확인
        </div>
    </div>
    """
    
    # 툴팁 내용 (마우스 오버 시)
    tooltip_text = f"🏆 TOP {rank}: {row['상권_코드_명']}<br>📊 적합도: {row['곱창집_적합도_v2']:.1f}점<br>🌙 야간유동: {row['T1_야간유동인구']:,}명"
    
    # 마커 추가
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['marker_size'],
        popup=folium.Popup(popup_html, max_width=350),
        color='black',
        weight=2,
        fillColor=row['color'],
        fillOpacity=0.85,
        tooltip=folium.Tooltip(tooltip_text, sticky=True)
    ).add_to(m)

print(f"✅ 마커 추가 완료: {len(map_data)}개")

# 범례 추가
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 190px; height: 160px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:12px; padding: 12px;
     box-shadow: 0 0 15px rgba(0,0,0,0.3);
     border-radius: 8px;">
     
<h4 style="margin: 0 0 12px 0; text-align: center; color: #333;">🍖 곱창집 적합도 등급</h4>
<div style="line-height: 1.8;">
    <div style="margin: 4px 0;">
        <i style="background:red; width:14px; height:14px; float:left; margin:3px 8px 0 0; border-radius:50%; border: 1px solid #999;"></i>
        <span style="font-weight: bold;">최우수</span> (40점 이상)
    </div>
    <div style="margin: 4px 0;">
        <i style="background:orange; width:14px; height:14px; float:left; margin:3px 8px 0 0; border-radius:50%; border: 1px solid #999;"></i>
        <span style="font-weight: bold;">우수</span> (35-40점)
    </div>
    <div style="margin: 4px 0;">
        <i style="background:yellow; width:14px; height:14px; float:left; margin:3px 8px 0 0; border-radius:50%; border: 1px solid #999;"></i>
        <span style="font-weight: bold;">양호</span> (30-35점)
    </div>
    <div style="margin: 4px 0;">
        <i style="background:green; width:14px; height:14px; float:left; margin:3px 8px 0 0; border-radius:50%; border: 1px solid #999;"></i>
        <span style="font-weight: bold;">보통</span> (25-30점)
    </div>
    <div style="margin: 4px 0;">
        <i style="background:blue; width:14px; height:14px; float:left; margin:3px 8px 0 0; border-radius:50%; border: 1px solid #999;"></i>
        <span style="font-weight: bold;">기타</span> (25점 미만)
    </div>
</div>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

print("✅ 범례 추가 완료")

print("=" * 70)

# 📊 지역별 분포 분석
# ================================================================
print("\n📊 지역별 분포 분석")
print("-" * 15)

def analyze_geographic_distribution():
    """지리적 분포 분석"""
    
    # 주요 권역별 키워드 정의
    regions = {
        '강남권': ['강남', '서초', '역삼', '선릉', '신논현', '교대'],
        '홍대권': ['홍대', '연남', '망리단', '신촌'],
        '강북권': ['노원', '수유', '불광'],
        '중구권': ['명동', '종로', '북창동'],
        '영등포권': ['영등포', '여의도', '노량진'],
        '관악권': ['신림', '총신대', '서울대'],
        '동대문권': ['대학로', '혜화'],
        '기타권역': []  # 위에 해당하지 않는 지역들
    }
    
    # 각 상권을 권역별로 분류
    region_data = {region: {'count': 0, 'locations': [], 'scores': []} for region in regions.keys()}
    
    for idx, row in map_data.iterrows():
        location_name = row['상권_코드_명']
        score = row['곱창집_적합도_v2']
        assigned = False
        
        # 권역별 키워드 매칭
        for region, keywords in regions.items():
            if region == '기타권역':
                continue
            if any(keyword in location_name for keyword in keywords):
                region_data[region]['count'] += 1
                region_data[region]['locations'].append(location_name)
                region_data[region]['scores'].append(score)
                assigned = True
                break
        
        # 매칭되지 않은 경우 기타권역에 추가
        if not assigned:
            region_data['기타권역']['count'] += 1
            region_data['기타권역']['locations'].append(location_name)
            region_data['기타권역']['scores'].append(score)
    
    print("📍 권역별 TOP 30 분포:")
    for region, data in region_data.items():
        if data['count'] > 0:
            avg_score = np.mean(data['scores']) if data['scores'] else 0
            print(f"   🏢 {region}: {data['count']}개 (평균 {avg_score:.1f}점)")
            if data['count'] <= 3:  # 3개 이하면 상권명도 표시
                for loc in data['locations']:
                    print(f"      • {loc}")
    
    return region_data

region_analysis = analyze_geographic_distribution()

print("=" * 70)

# 📊 추가 시각화
# ================================================================
print("\n📊 추가 시각화 생성")
print("-" * 12)

# 1. 적합도 점수별 산점도 (위도 vs 경도)
fig_scatter = px.scatter(
    map_data,
    x='longitude',
    y='latitude', 
    size='곱창집_적합도_v2',
    color='곱창집_적합도_v2',
    hover_name='상권_코드_명',
    hover_data={
        'T1_야간유동인구': ':,',
        '한식_월매출': ':,.0f',
        'longitude': False,
        'latitude': False
    },
    title="🗺️ TOP 30 곱창집 입지 지리적 분포",
    labels={
        'longitude': '경도 (Longitude)',
        'latitude': '위도 (Latitude)',
        '곱창집_적합도_v2': '적합도 점수'
    },
    color_continuous_scale='Reds',
    size_max=25
)

fig_scatter.update_layout(
    font=dict(family="Arial", size=12),
    height=500,
    showlegend=True,
    title_x=0.5
)

fig_scatter.show()

# 2. 적합도 등급별 파이 차트
grade_counts = map_data['color'].value_counts()
grade_labels = [color_info[color] for color in grade_counts.index]

fig_pie = px.pie(
    values=grade_counts.values,
    names=grade_labels,
    title="🏆 TOP 30 적합도 등급 분포",
    color_discrete_map={
        '최우수 (40점 이상)': '#ff4444',
        '우수 (35-40점)': '#ff8800', 
        '양호 (30-35점)': '#ffdd00',
        '보통 (25-30점)': '#44aa44',
        '기타 (25점 미만)': '#4488ff'
    }
)

fig_pie.update_traces(textposition='inside', textinfo='percent+label')
fig_pie.update_layout(
    font=dict(family="Arial", size=12),
    height=500,
    title_x=0.5
)

fig_pie.show()

# 3. TOP 10 상권 바 차트
top10_data = map_data.head(10)
fig_bar = px.bar(
    top10_data,
    x='상권_코드_명',
    y='곱창집_적합도_v2',
    color='곱창집_적합도_v2',
    title="🥇 곱창집 입지 TOP 10 적합도 점수",
    labels={
        '상권_코드_명': '상권명',
        '곱창집_적합도_v2': '적합도 점수'
    },
    color_continuous_scale='Reds'
)

fig_bar.update_layout(
    font=dict(family="Arial", size=12),
    height=500,
    xaxis={'tickangle': 45},
    title_x=0.5
)

fig_bar.show()

print("=" * 70)

# 💾 결과 저장
# ================================================================
print("\n💾 결과 저장")
print("-" * 8)

try:
    # HTML 지도 저장
    map_file = OUTPUT_PATH / "gopchang_location_map.html"
    m.save(str(map_file))
    print(f"✅ 인터랙티브 지도: {map_file.name}")
    
    # 좌표 포함 데이터 저장
    coords_output = OUTPUT_PATH / "gopchang_top30_coordinates.csv"
    map_data.to_csv(coords_output, index=False, encoding='utf-8-sig')
    print(f"✅ 좌표 데이터: {coords_output.name}")
    
    # 권역별 분석 결과 저장
    region_report = OUTPUT_PATH / "gopchang_regional_analysis.txt"
    with open(region_report, 'w', encoding='utf-8') as f:
        f.write("🗺️ 곱창집 입지 권역별 분석 보고서\n")
        f.write(f"📅 분석일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("=" * 50 + "\n\n")
        
        f.write("📊 권역별 분포 현황:\n")
        for region, data in region_analysis.items():
            if data['count'] > 0:
                avg_score = np.mean(data['scores']) if data['scores'] else 0
                f.write(f"• {region}: {data['count']}개 상권 (평균 {avg_score:.1f}점)\n")
                for loc in data['locations']:
                    f.write(f"  - {loc}\n")
                f.write("\n")
        
        f.write("🎯 지역별 특성 분석:\n")
        f.write("• 강남권: 높은 매출 + 프리미엄 상권\n")
        f.write("• 홍대권: 젊은층 + 주류친화 문화\n")
        f.write("• 중구권: 관광객 + 전통 상권\n")
        f.write("• 강북권: 지역 거점 + 안정적 수요\n")
    
    print(f"✅ 권역별 분석: {region_report.name}")
    
except Exception as e:
    print(f"❌ 저장 중 오류: {e}")

print("=" * 70)

# 🎯 완료 요약
# ================================================================
print("\n🎯 지도 시각화 완료!")
print("-" * 15)

end_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"✅ 완료 시간: {end_time}")

print(f"\n📊 지도 시각화 성과:")
print(f"   🗺️ TOP {len(map_data)}개 상권 100% 지도 표시")
print(f"   🎨 5단계 색상 등급 (적합도별)")
print(f"   📋 상세 팝업 정보 (6개 핵심 지표)")
print(f"   📍 권역별 분포 분석 완료")

print(f"\n🔍 핵심 발견 사항:")
strongest_region = max(region_analysis.items(), 
                      key=lambda x: np.mean(x[1]['scores']) if x[1]['scores'] else 0)
print(f"   • 최강 권역: {strongest_region[0]} (평균 {np.mean(strongest_region[1]['scores']):.1f}점)")
print(f"   • 상위권 집중: {len([x for x in map_data['곱창집_적합도_v2'] if x >= 35])}개 상권이 35점 이상")
print(f"   • 지역 다양성: {len([r for r, d in region_analysis.items() if d['count'] > 0])}개 권역에 분포")

print(f"\n📁 생성된 파일:")
print(f"   🌐 gopchang_location_map.html - 인터랙티브 지도")
print(f"   📊 gopchang_top30_coordinates.csv - 좌표 데이터")
print(f"   📋 gopchang_regional_analysis.txt - 권역별 분석")

print(f"\n🚀 지도 사용 방법:")
print(f"   1. gopchang_location_map.html 파일을 브라우저에서 열기")
print(f"   2. 마커 클릭하여 상세 정보 확인")
print(f"   3. 지도 확대/축소로 지역별 클러스터 관찰")
print(f"   4. 범례 참고하여 등급별 상권 구분")

print(f"\n🎯 다음 단계: B. TOP 10 상권 세부 분석")
print(f"   • 상권별 강점/약점 심층 분석")
print(f"   • 경쟁 환경 및 타겟 고객 분석")
print(f"   • 곱창집 최적 컨셉 제안")

print("=" * 70)
print("🗺️ 지도 시각화 완료!")
print("🎯 이제 브라우저에서 인터랙티브 지도를 확인할 수 있습니다!")

In [ ]:
map_data.head